In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from utils import chart_price, plot, perf, v_backtester, c_backtester, perf_var, summary
import sys
sys.path.append('/home/tomek/ib_tools')
import matplotlib.pyplot as plt
%matplotlib inline
from tester import run, simulate, calibrate, calibrate_multiple

In [2]:
def ema(data, periods, smooth, vol):
    data = data.copy()
    data['ema_fast'] = data.close.ewm(span=periods).mean()
    data['ema_slow'] = data.close.ewm(span=periods*3).mean()
    data['ema_diff'] = data.ema_fast - data.ema_slow
    data['ema_diff_norm'] = data.ema_diff / vol 
    #data.ema_diff.ewm(span=vol_lookback).std()
    #data['ema_diff_norm'] = data.ema_diff / data.ema_diff.rolling(periods).std()
    #data[f'ema_{periods}'] = data['ema_diff_norm'].ewm(span=smooth).mean()
    data[f'ema_{periods}'] = data['ema_diff_norm']
    return data[f'ema_{periods}']

In [3]:
def reducer(x):
    if x < -2.7:
        return max(-(np.log(np.abs(x) -2.7) - .5), 0)
    elif x > 2.7:
        return min((np.log(np.abs(x) -2.7) - .5), 0)
    else:
        return - x**3

In [4]:
def bolli(data, periods, smooth, vol):
    data = data.copy()
    data['mid'] = data.close.ewm(span=periods).mean()
    data['distance'] = data.mid - data.close
    data['std'] = data.close.ewm(span=periods).std()
    data['distance_scaled'] = data.distance / data['std']
    data[f'bolli_{periods}'] = data['distance_scaled']
    return data[f'bolli_{periods}']

In [5]:
def bolli_reduced(data, periods, smooth, vol):
    data = data.copy()
    #data[f'bolli_reduced_{periods}'] = smooth(bolli(data, periods, smooth, vol).apply(reducer), periods/3)
    data[f'bolli_reduced_{periods}'] = bolli(data, periods, smooth, vol).apply(reducer)
    return data[f'bolli_reduced_{periods}']

In [6]:
def breakout(data, periods, smooth, vol):
    up  = (data.close >= data.close.rolling(periods).max()) * 1
    down = (data.close <= data.close.rolling(periods).min()) * 1
    return (up - down).ewm(span=max((int(periods/3), 1))).mean().rename(f'break_{periods}', inplace=True)

In [7]:
def carver(data, periods, smooth, vol):
    data = data.copy()
    data['max'] = data.close.rolling(periods).max()
    data['min'] = data.close.rolling(periods).min()
    data['mid'] = data[['min', 'max']].mean(axis=1)
    data['breakout'] = data['close'] - data['mid']
    data['breakout_norm'] = data['breakout'] / (data['max'] - data['min'])
    data[f'carver_{periods}'] = smooth(data['breakout_norm'], periods/3)
    return data[f'carver_{periods}']

In [8]:
def roc(data, periods, smooth, vol):
    data = data.copy()
    data['roc'] = data.close.pct_change(periods) * 1000
    #data['std'] = data['roc'].ewm(span=200).std()
    #data['roc_normalized'] = data['roc'] / data['std']
    data['roc_normalized'] = data['roc'] / vol
    #data[f'roc_{periods}'] = smooth(data['roc_normalized'], periods/3)
    data[f'roc_{periods}'] = data['roc_normalized']
    return data[f'roc_{periods}']

In [9]:
data = calibrate_multiple('ES', roc)

In [13]:
weights = pd.DataFrame()
adjustments = pd.DataFrame()
multipliers = pd.Series()
for symbol in ['NQ', 'ES', 'GC', 'CL']:
    w,a,m = calibrate_multiple(symbol, roc)
    weights[symbol] = w
    adjustments[symbol] = a
    multipliers[symbol] = m
weights, adjustments, multipliers

(               NQ        ES        GC        CL
 roc_5    0.176056  0.172632  0.181185  0.177014
 roc_10   0.152383  0.151518  0.160209  0.152213
 roc_20   0.161225  0.165106  0.147569  0.149765
 roc_40   0.172758  0.158756  0.166146  0.157358
 roc_80   0.169972  0.180685  0.156992  0.182411
 roc_160  0.167605  0.171303  0.187898  0.181240,
                 NQ         ES        GC        CL
 roc_5    48.660678  16.802580  7.344630  0.331112
 roc_10   39.648234  14.943605  5.358343  0.227391
 roc_20   30.965184   9.439575  3.567845  0.160122
 roc_40   21.901081   9.157381  2.999457  0.148428
 roc_80   18.721242   6.585801  2.629839  0.108833
 roc_160   7.285728   2.965453  2.053643  0.063916,
 NQ    1.389199
 ES    1.409544
 GC    1.402660
 CL    1.307953
 dtype: float64)

In [14]:
1/6

0.16666666666666666

In [16]:
from tester import get_data, get_candles, get_avg_vol, get_vol

In [17]:
data = get_data('ES', '20180901', '20191231')

In [18]:
data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-09-03 00:00:00,2906.50,2907.00,2905.75,2905.75,949,2906.375,161
2018-09-03 00:01:00,2905.75,2906.25,2905.75,2905.75,329,2906.125,80
2018-09-03 00:02:00,2905.75,2906.00,2905.75,2906.00,53,2905.900,19
2018-09-03 00:03:00,2906.25,2906.75,2906.25,2906.75,145,2906.450,49
2018-09-03 00:04:00,2906.50,2906.75,2906.00,2906.00,135,2906.300,46
...,...,...,...,...,...,...,...
2019-09-05 08:51:00,2957.50,2957.75,2957.50,2957.50,361,2957.525,70
2019-09-05 08:52:00,2957.50,2958.00,2957.00,2957.25,517,2957.575,112
2019-09-05 08:53:00,2957.50,2957.75,2957.25,2957.50,267,2957.400,85


In [22]:
months = [g for n, g in data.groupby(pd.Grouper(freq='M'))]

In [ ]:
months

In [ ]:
months[9]

In [19]:
candles = get_candles(data, get_avg_vol(data, 120)).set_index('date')

In [20]:
candles

,open,high,low,close,barCount,volume
date,,,,,,
2018-09-04 02:31:00,2906.50,2912.50,2902.50,2904.50,34605,134452
2018-09-04 13:31:00,2904.50,2912.00,2894.75,2896.00,28532,134107
2018-09-04 15:34:00,2896.00,2898.50,2891.75,2896.00,16732,130583
2018-09-04 15:50:00,2896.00,2896.25,2886.25,2886.50,19416,144627
2018-09-04 16:18:00,2886.75,2898.25,2886.50,2896.50,17426,139397
...,...,...,...,...,...,...
2019-09-04 19:03:00,2928.00,2934.50,2927.75,2933.75,22662,136899
2019-09-04 21:07:00,2934.00,2935.50,2929.00,2935.00,20499,134818
2019-09-04 21:58:00,2935.00,2939.00,2934.50,2937.75,15157,123074


In [23]:
start = months[0].index[0]
end = months[0].index[-1]

In [24]:
start, end

(Timestamp('2018-09-03 00:00:00'), Timestamp('2018-09-28 22:59:00'))

In [25]:
period = candles.loc[start:end]

In [26]:
period

,open,high,low,close,barCount,volume
date,,,,,,
2018-09-04 02:31:00,2906.50,2912.50,2902.50,2904.50,34605,134452
2018-09-04 13:31:00,2904.50,2912.00,2894.75,2896.00,28532,134107
2018-09-04 15:34:00,2896.00,2898.50,2891.75,2896.00,16732,130583
2018-09-04 15:50:00,2896.00,2896.25,2886.25,2886.50,19416,144627
2018-09-04 16:18:00,2886.75,2898.25,2886.50,2896.50,17426,139397
...,...,...,...,...,...,...
2018-09-28 17:59:00,2921.50,2925.50,2921.25,2921.75,16542,138491
2018-09-28 19:42:00,2921.50,2923.75,2915.50,2922.75,16539,135231
2018-09-28 21:11:00,2922.75,2924.00,2917.50,2918.25,15378,133601


In [35]:
len(period.resample('D').last().dropna())

19

In [36]:
165/19

8.68421052631579

In [27]:
vols = get_vol(candles, 200)

In [32]:
vols.iloc[150:].head(50)

date
2018-09-27 16:07:00         NaN
2018-09-27 16:58:00         NaN
2018-09-27 18:17:00         NaN
2018-09-27 20:06:00         NaN
2018-09-27 21:12:00         NaN
2018-09-27 21:58:00         NaN
2018-09-28 09:56:00         NaN
2018-09-28 14:51:00         NaN
2018-09-28 15:57:00         NaN
2018-09-28 16:46:00         NaN
2018-09-28 17:59:00         NaN
2018-09-28 19:42:00         NaN
2018-09-28 21:11:00         NaN
2018-09-28 21:43:00         NaN
2018-09-28 21:58:00         NaN
2018-10-01 04:06:00         NaN
2018-10-01 14:38:00         NaN
2018-10-01 15:51:00         NaN
2018-10-01 16:39:00         NaN
2018-10-01 17:57:00         NaN
2018-10-01 19:15:00         NaN
2018-10-01 20:45:00         NaN
2018-10-01 21:34:00         NaN
2018-10-01 21:58:00         NaN
2018-10-02 09:17:00         NaN
2018-10-02 15:29:00         NaN
2018-10-02 16:02:00         NaN
2018-10-02 16:51:00         NaN
2018-10-02 18:02:00         NaN
2018-10-02 19:46:00         NaN
2018-10-02 21:12:00    4.702400
201

In [ ]:
r= roc(period, 160, None, vols)


In [ ]:
r.iloc[50:].head(50)

In [ ]:
r.dropna()

In [ ]:
r.tail(50)

In [ ]:
data = period.copy()
data['roc'] = data.close.pct_change(5) * 1000
data.head(50)

In [ ]:
data['vol'] = vols
data['roc_normalized'] = data['roc'] / vols
data

In [ ]:
vols.head(50)

In [ ]:
x = pd.Series({'a': 1, 'b': -2, 'c': 4, 'd': -7})
x

In [ ]:
x.apply(lambda x: max(x,0))